### to scrape:
- Trends in CogSci
- DONE Topics in CogSci
- Cognition
- Proceedings to CogSci
- DONE ARTHUR Cognitive Science

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-colorblind')
plt.rcParams['image.cmap'] = 'RdBu'
import numpy as np
# Import custom code
import csv
from bs4 import BeautifulSoup
import urllib
import re
import string

### scraping Topics in CogSci

In [177]:
#collect all issue links
all_issues_url = []
for year in range(2009,2017):
    landing_url = urllib.urlopen('http://onlinelibrary.wiley.com/journal/10.1111/(ISSN)1756-8765/issues?year='+str(year)).read()
    soup = BeautifulSoup(landing_url, 'html.parser')
    issues = soup.find_all("span", {"class": "issue-number"})
    for issue in issues:
        all_issues_url.append('http://onlinelibrary.wiley.com' + str(issue.find("a")['href']))

#collect all article links
all_article_url = []
all_year = []
for issue in all_issues_url:
    soup = BeautifulSoup(urllib.urlopen(issue), 'html.parser')
    articles = soup.find_all("div", {"class": 'citation tocArticle'})
    for article in articles:
        all_article_url.append('http://onlinelibrary.wiley.com'+article.find_all("a")[0]['href'])
        all_year.append(issue[48:52])

In [144]:
# collect information from article links
all_authors = []
all_affli = []
cnt = 0
for article in all_article_url:
    soup = BeautifulSoup(urllib.urlopen(article), 'html.parser')
    author_ele = soup.find_all('h3', {'class': 'article-header__authors-name'})
    affli_ele = soup.find_all('ol', {'class': 'article-header__authors-item-aff-addr'})
    authors = [cur_aut.text.split(',')[0].encode('utf-8') for cur_aut in author_ele]
    affli = [cur_aff.text.encode('utf-8') for cur_aff in affli_ele]
    all_authors.append(authors)
    all_affli.append(affli)
    cnt+=1
    if not np.mod(cnt,15):
        print cnt/450.,

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450


In [187]:
with open('topiCS_authors.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for author in all_authors:
        if author != []:
            writer.writerow(author)

with open('topiCS_affiliations.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for idx, affli in enumerate(all_affli):
        if affli != []:
            row = [all_year[idx]]
            for aff in affli:
                row.append(aff.lstrip())
            
            writer.writerow(row)

### Scrape MIT BCS

In [54]:
# get MIT BCS faculty and pages
landing_url = 'https://bcs.mit.edu/our-faculty'
soup = BeautifulSoup(urllib.urlopen(landing_url), 'html.parser')
PI_objs = soup.find_all('div', {'class': 'field user-full-name-link'})
all_PI = [obj.text.encode('utf-8') for obj in PI_objs]
all_links = ['https://bcs.mit.edu'+obj.find('a')['href'].encode('utf-8') for obj in PI_objs]

#get condensed publication list
all_pubs = []
for idx, link in enumerate(all_links):
    print all_PI[idx], '...',
    soup = BeautifulSoup(urllib.urlopen(link), 'html.parser')
    pubs_obj = soup.find('div', {'class': 'field field-name-field-publications field-type-text-long field-label-hidden view-mode-full'})
    if pubs_obj is not None:
        pubs = pubs_obj.find_all('p')
        all_pubs.append([pub.text.encode('utf-8') for pub in pubs])
    else:
        all_pubs.append([])
        print 'No Pubs.',

 Adelson, Edward ... Bear, Mark ... Bizzi, Emilio ... Boyden, Ed ... Brown, Emery ... Choi, Gloria ... Chung, Kwanghun ... Constantine-Paton, Martha ... Desimone, Robert ... DiCarlo, James ... Fee, Michale ... Feng, Guoping ... Flavell, Steven ... Gabrieli, John ... Gibson, Ted ... Graybiel, Ann ... Harnett, Mark ... Heiman, Myriam ... Hein, Alan ... Held, Richard ... Hockfield, Susan ... Hogan, Neville ... Horvitz, H. Robert ... Jasanoff, Alan ... Jazayeri, Mehrdad ... Kanwisher, Nancy ... Levy, Roger ... Lin, Yingxi ... Littleton, Troy ... McDermott, Josh ... Miller, Earl ... Nedivi, Elly ... Poggio, Tomaso ... Potter, Mary ... Prelec, Drazen ... Quinn, William ... Rokers, Bas ... Rosasco, Lorenzo ... No Pubs. Saxe, Rebecca ... Schiller, Peter ... Schneider, Gerald ... Schulz, Laura ... Sinha, Pawan ... Slotine, Jean-Jacques ... Sur, Mriganka ... Tenenbaum, Joshua ... Tonegawa, Susumu ... Tsai, Li-Huei ... Tye, Kay ... Ullman, Shimon ... Wexler, Ken ... Wilson, Matt ... Wurtman, Rich

In [61]:
with open('MIT_BCS.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for idx, PI in enumerate(all_PI):
        row = [PI]
        for pub in all_pubs[idx]:
            row.append(pub)
        writer.writerow(row)


### scrape Rochester BCS

In [2]:
landing_url = 'http://www.sas.rochester.edu/bcs/people/faculty/index.html'
soup = BeautifulSoup(urllib.urlopen(landing_url), 'html.parser')
PI_objs = soup.find_all('h4')
all_PI = [obj.text.rstrip().encode('utf-8') for obj in PI_objs]


In [3]:
all_links = []
for obj in PI_objs:
    if obj.find('a') is not None:
        all_links.append('http://www.sas.rochester.edu/bcs/people/faculty/'+obj.find('a')['href'].encode('utf-8'))
    else:
        all_links.append('')

In [11]:
for idx_link, link in enumerate(all_links[:2]):
    if link is not '':
        soup = BeautifulSoup(urllib.urlopen(link), 'html.parser')
        headers = soup.find_all('h3')
        if headers is not []:
            for idx_hdr, hdr in enumerate(headers):
                if hdr.text == 'Selected Publications':
                    print idx_hdr

0
1


In [15]:
soup.find_all('ul')

[<ul class="title-area show-for-small-only"><li class="name"><h1><a href="/bcs/index.html">Home</a></h1></li><li class="has-form"><form action="http://www.rochester.edu/college/search/" id="search"><input name="cx" type="hidden" value="009288150455229766548:c2f8fereqgm"/><input name="cof" type="hidden" value="FORID:10"/><input name="ie" type="hidden" value="UTF-8"/><div class="row collapse"><div class="small-10 columns"><input accesskey="?" class="search_box" id="search" name="q" onfocus="this.value=''" placeholder="SEARCH ROCHESTER.EDU" type="text" value="SEARCH ROCHESTER.EDU"/></div><div class="small-2 columns"><input class="button" type="submit" value="Go"/></div></div></form></li><li class="toggle-topbar menu-icon"><a href="#">Menu</a></li></ul>,
 <ul><li><a href="/bcs/index.html"><img alt="HOME" class="home-icon" src="//www.rochester.edu/college/assets/cinnamon/images/icons/home.png"/><span class="show-for-small-only">Home</span></a></li><li class="divider"></li><li class="has-dro